# സെമാന്റിക് കർണൽ

ഈ കോഡ് സാമ്പിളിൽ, [സെമാന്റിക് കർണൽ](https://aka.ms/ai-agents-beginners/semantic-kernel) AI ഫ്രെയിംവർക്കിനെ ഉപയോഗിച്ച് ഒരു അടിസ്ഥാന ഏജന്റ് സൃഷ്ടിക്കും.

വിവിധ ഏജന്റിക് പാറ്റേണുകൾ നടപ്പിലാക്കുമ്പോൾ, പിന്നീട് ഉപയോഗിക്കുന്ന അധിക കോഡ് സാമ്പിളുകളിൽ ഞങ്ങൾ സ്വീകരിക്കുന്ന ഘട്ടങ്ങൾ നിങ്ങൾക്ക് കാണിക്കാൻ ഈ സാമ്പിളിന്റെ ലക്ഷ്യമാണ്.


## ആവശ്യമായ പൈത്തൺ പാക്കേജുകൾ ഇറക്കുമതി ചെയ്യുക


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## ക്ലയന്റ് സൃഷ്ടിക്കൽ

ഈ ഉദാഹരണത്തിൽ, LLM-ലേക്ക് പ്രവേശനത്തിനായി [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ഉപയോഗിക്കും.

`ai_model_id` എന്നത് `gpt-4o-mini` ആയി നിർവചിച്ചിരിക്കുന്നു. GitHub Models മാർക്കറ്റ്പ്ലേസിൽ ലഭ്യമായ മറ്റൊരു മോഡലിലേക്ക് മോഡൽ മാറ്റി വ്യത്യസ്ത ഫലങ്ങൾ കാണാൻ ശ്രമിക്കുക.

GitHub Models-ക്കുള്ള `base_url` ഉപയോഗിക്കുന്ന `Azure Inference SDK` ഉപയോഗിക്കാൻ, Semantic Kernel-ൽ `OpenAIChatCompletion` കണക്റ്റർ ഉപയോഗിക്കും. Semantic Kernel ഉപയോഗിച്ച് മറ്റ് മോഡൽ പ്രൊവൈഡർമാർക്കുള്ള [ലഭ്യമായ മറ്റ് കണക്റ്ററുകൾ](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) ഉണ്ട്.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## ഏജന്റ് സൃഷ്ടിക്കുന്നത്

താഴെ `TravelAgent` എന്ന ഏജന്റ് സൃഷ്ടിക്കുന്നു.

ഈ ഉദാഹരണത്തിൽ, വളരെ ലളിതമായ നിർദ്ദേശങ്ങൾ ഉപയോഗിക്കുന്നു. ഏജന്റിന്റെ പ്രതികരണം എങ്ങനെ വ്യത്യസ്തമാകുന്നുവെന്ന് കാണാൻ നിങ്ങൾക്ക് ഈ നിർദ്ദേശങ്ങൾ മാറ്റാം.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## ഏജന്റ് പ്രവർത്തിപ്പിക്കൽ

ഇപ്പോൾ, `ChatHistoryAgentThread` എന്ന തരം ഒരു ത്രെഡ് നിർവചിച്ച് ഏജന്റ് പ്രവർത്തിപ്പിക്കാം. ഏജന്റിന്റെ invoke_stream `messages` കീവേഡ് ആർഗുമെന്റിലേക്ക് ആവശ്യമായ സിസ്റ്റം സന്ദേശങ്ങൾ നൽകാം.

ഇവ നിർവചിച്ചതിന് ശേഷം, ഉപയോക്താവ് ഏജന്റിലേക്ക് അയക്കുന്ന സന്ദേശമായ `user_inputs` സൃഷ്ടിക്കുന്നു. ഈ സന്ദേശം `Plan me a sunny vacation` ആയി സജ്ജമാക്കിയിരിക്കുന്നു.

ഏജന്റ് വ്യത്യസ്തമായി പ്രതികരിക്കുന്നതിനെ കാണാൻ ഈ സന്ദേശം മാറ്റാൻ മടിക്കേണ്ടതില്ല.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസത്യവാദം**:  
ഈ രേഖ AI വിവർത്തന സേവനമായ [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. കൃത്യതയ്ക്കായി ഞങ്ങൾ ശ്രമിക്കുന്നുവെങ്കിലും, ഓട്ടോമേറ്റഡ് വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റായ വിവരങ്ങൾ ഉണ്ടാകാൻ സാധ്യതയുണ്ട്. അതിന്റെ സ്വാഭാവിക ഭാഷയിലുള്ള മൗലിക രേഖ പ്രാമാണികമായ ഉറവിടമായി പരിഗണിക്കണം. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യുന്നു. ഈ വിവർത്തനം ഉപയോഗിച്ച് ഉണ്ടാകുന്ന തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
